
# Profitable UK App Profiles for the App Store and Google Play Markets

The aim of the project is to find mobile app profiles that are profitable for the App Store and Google Play markets. Based on the project results, team of developers will be able to make data-driven decisions with respect to the kind of apps they build.

At our company, we only build apps that are free to download and install, and our main source of revenue consists of in-app ads. This means that our revenue for any given app is mostly influenced by the number of users that use our app. Our goal for this project is to analyze data to help our developers understand what kinds of apps are likely to attract more users.

***As of September 2018, there were approximately 2 million iOS apps available on the App Store, and 2.1 million Android apps on Google Play.***

Collecting data for over four million apps requires a significant amount of time and money. Luckily, there are two data sets that seem suitable for this purpose: 

- [A data set](https://www.kaggle.com/lava18/google-play-store-apps/home) containing data about approximately ten thousand Android apps from Google Play
- [A data set](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps/home) containing data about approximately seven thousand iOS apps from the App Store

---

Let's start by opening the two data sets and then continue with exploring the data.


In [1]:
from csv import reader

### Read Apple Store dataset ###
opened_file = open('data/AppleStore.csv', encoding='utf8')
read_file = reader(opened_file)
apple = list(read_file)
apple_header = apple[0]
apple_data = apple[1:]

### Read Google Play dataset ###
opened_file = open('data/googleplaystore.csv', encoding='utf8')
read_file = reader(opened_file)
google = list(read_file)
google_header = google[0]
google_data = google[1:]

Let's create a function that will give us an overview of datasets we use. 

In [2]:
def explore_data(market, header, data):
    print("*** " + market + " ***")
    print("Headers:")
    print(header)
    print("Data:")
    print(data[0:6])
    print("Number of rows: " + str(len(data)))
    print("Number of columns: " + str(len(data[0])))

And print those

In [3]:
explore_data("Apple Store", apple_header, apple_data)
explore_data("Google Play Store", google_header, google_data)

*** Apple Store ***
Headers:
['', 'id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']
Data:
[['1', '281656475', 'PAC-MAN Premium', '100788224', 'USD', '3.99', '21292', '26', '4', '4.5', '6.3.5', '4+', 'Games', '38', '5', '10', '1'], ['2', '281796108', 'Evernote - stay organized', '158578688', 'USD', '0', '161065', '26', '4', '3.5', '8.2.2', '4+', 'Productivity', '37', '5', '23', '1'], ['3', '281940292', 'WeatherBug - Local Weather, Radar, Maps, Alerts', '100524032', 'USD', '0', '188583', '2822', '3.5', '4.5', '5.0.0', '4+', 'Weather', '37', '5', '3', '1'], ['4', '282614216', 'eBay: Best App to Buy, Sell, Save! Online Shopping', '128512000', 'USD', '0', '262241', '649', '4', '4.5', '5.10.0', '12+', 'Shopping', '37', '5', '9', '1'], ['5', '282935706', 'Bible', '92774400', 'USD', '0', '985920', '5320', '4.5', '5', '7

## Data cleaning

According to kaggle forum in google dataset the row at index 10473 *(In our case 10472 as we seperated headers)* contains an error. Below we can see that the rating for **Life Made WI-Fi Touchscreen Photo Frame** is 19 while for other records max is 5 and the category is 1.9. 

Let's remove that from the dataset.

In [4]:
print(google_header)
print(google_data[10472])
del(google_data[10472])

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']
['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


It seems that both datasets may contain duplicates. Let's check that:

In [5]:
def find_duplicates(dataset, title_index, market):
    unique_rows = []
    duplicate_rows = []
    for row in dataset:
        title = row[title_index]
        if title in unique_rows:
            duplicate_rows.append(title)
        else:
            unique_rows.append(title)
    print("Number of " + market + " duplicate rows:", len(duplicate_rows))
    print("Examples of " + market + " duplicate rows:", duplicate_rows[:5])
    
find_duplicates(google_data, 0, "Google")
find_duplicates(apple_data, 2, "Apple")

Number of Google duplicate rows: 1181
Examples of Google duplicate rows: ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings']
Number of Apple duplicate rows: 2
Examples of Apple duplicate rows: ['VR Roller Coaster', 'Mannequin Challenge']


Now we need to remove those duplicates. Criterion for removing duplicates will be the total number of reviews. It seems to appear that as higher is this number as as more recent record that is. In the result we must have two cleaned datasets with lengths of 9659 items for Google and 7195 for Apple.

In [6]:
def remove_duplicates(dataset, title_index, reviews_count_index,): 
    reviews_max = {}
    for row in dataset:
        title = row[title_index]
        reviews_count = int(row[reviews_count_index])
        if title not in reviews_max:
            reviews_max[title] = reviews_count
        else:
            if reviews_max[title] < reviews_count:
                reviews_max[title] = reviews_count
    
    dataset_clean = []
    already_added = []
    for row in dataset:
        title = row[title_index]
        reviews_count = int(row[reviews_count_index])
        if title in reviews_max and reviews_max[title] == reviews_count and title not in already_added:
            dataset_clean.append(row)
            already_added.append(title)
    
    return dataset_clean
        
    
google_data_clean = remove_duplicates(google_data, 0, 3)
apple_data_clean = remove_duplicates(apple_data, 2, 6)
print("Google number of items:", len(google_data_clean))
print("Apple number of items:", len(apple_data_clean))

Google number of items: 9659
Apple number of items: 7195


Based on the project requirements, we are intersted only in application for English market *(English title)*. 

The numbers corresponding to the characters we commonly use in an English text are all in the range 0 to 127, according to the ASCII (American Standard Code for Information Interchange) system. Based on this number range, we can build a function that detects whether a character belongs to the set of common English characters or not. If the number is equal to or less than 127, then the character belongs to the set of common English characters.

As far as titles of applications might contain emojis and other symbols out of 0-127 range, we will check if more than 3 symbols of the title are out of the range and if yes, we will exlude that from the list. 

In [7]:
def remove_foreign_apps(dataset, title_index):
    index = 0
    english_dataset = []
    for row in dataset:
        title = row[title_index]
        letter_count = 0
        for letter in title:
            if ord(letter) > 127:
                letter_count += 1
        if letter_count <= 3:
            english_dataset.append(dataset[index])
        index += 1
    return english_dataset
google_data_clean = remove_foreign_apps(google_data_clean, 0)
apple_data_clean = remove_foreign_apps(apple_data_clean, 2)

print("Google number of items:", len(google_data_clean))
print("Apple number of items:", len(apple_data_clean))

Google number of items: 9614
Apple number of items: 6181


As we mentioned in the introduction, we only build apps that are free to download and install, and our main source of revenue consists of in-app ads. Our data sets contain both free and non-free apps, and we'll need to isolate only the free apps for our analysis. Below, we isolate the free apps for both our data sets.

In [8]:
def remove_paid_apps(dataset, price_index, market):
    index = 0
    free_dataset = []
    for row in dataset:
        price = row[price_index]
        if price == '0':
            free_dataset.append(dataset[index])
        index += 1
    return free_dataset
    
google_data_clean = remove_paid_apps(google_data_clean, 7, "Google")
apple_data_clean = remove_paid_apps(apple_data_clean, 5, "Apple")
print("Google number of items:", len(google_data_clean))
print("Apple number of items:", len(apple_data_clean))

Google number of items: 8864
Apple number of items: 3220


We're left with 8864 Android apps and 3220 iOS apps, which should be enough for our analysis.

## Most common apps by genre

As we mentioned in the introduction, our aim is to determine the kinds of apps that are likely to attract more users because our revenue is highly influenced by the number of people using our apps.

To minimize risks and overhead, our validation strategy for an app idea is comprised of three steps:

1. Build a minimal Android version of the app, and add it to Google Play.
2. If the app has a good response from users, we then develop it further.
3. If the app is profitable after six months, we also build an iOS version of the app and add it to the App Store.

Because our end goal is to add the app on both the App Store and Google Play, we need to find app profiles that are successful on both markets. For instance, a profile that might work well for both markets might be a productivity app that makes use of gamification.

Let's begin the analysis by getting a sense of the most common genres for each market. For this, we'll build a frequency table for the prime_genre column of the App Store data set, and the Genres and Category columns of the Google Play data set.

In [13]:
def freq_table(dataset, index):
    table = {}
    total = 0
    for row in dataset:
        item = row[index]
        if item in table:
            table[item] += 1
        else: 
            table[item] = 1
        total += 1
    
    table_percent = {}
    for key in table: 
        percent = table[key]/total * 100
        table_percent[key] = percent
    
    return table_percent
        
def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])      

And let's print those frequency tables. 

Let's print IOS prime_genre table firstly

In [67]:
display_table(apple_data_clean, -5)

Games : 58.13664596273293
Entertainment : 7.888198757763975
Photo & Video : 4.968944099378882
Education : 3.6645962732919255
Social Networking : 3.291925465838509
Shopping : 2.608695652173913
Utilities : 2.515527950310559
Sports : 2.142857142857143
Music : 2.049689440993789
Health & Fitness : 2.018633540372671
Productivity : 1.7391304347826086
Lifestyle : 1.5838509316770186
News : 1.3354037267080745
Travel : 1.2422360248447204
Finance : 1.1180124223602486
Weather : 0.8695652173913043
Food & Drink : 0.8074534161490683
Reference : 0.5590062111801243
Business : 0.5279503105590062
Book : 0.43478260869565216
Navigation : 0.18633540372670807
Medical : 0.18633540372670807
Catalogs : 0.12422360248447205


We can see that among the free English apps, more than a half (58.13%) are games. Entertainment apps are close to 8%, followed by photo and video apps, which are close to 5%. Only 3.66% of the apps are designed for education, followed by social networking apps which amount for 3.29% of the apps in our data set.

The general impression is that App Store (at least the part containing free English apps) is dominated by apps that are designed for fun (games, entertainment, photo and video, social networking, sports, music, etc.), while apps with practical purposes (education, shopping, utilities, productivity, lifestyle, etc.) are more rare. However, the fact that fun apps are the most numerous doesn't also imply that they also have the greatest number of users — the demand might not be the same as the offer.

Let's continue by examining the Genres and Category columns of the Google Play data set (two columns which seem to be related).

Android Category:

In [17]:
display_table(google_data_clean, 1) 

FAMILY : 18.907942238267147
GAME : 9.724729241877256
TOOLS : 8.461191335740072
BUSINESS : 4.591606498194946
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.7003610108303246
MEDICAL : 3.531137184115524
SPORTS : 3.395758122743682
PERSONALIZATION : 3.3167870036101084
COMMUNICATION : 3.2378158844765346
HEALTH_AND_FITNESS : 3.0798736462093865
PHOTOGRAPHY : 2.944494584837545
NEWS_AND_MAGAZINES : 2.7978339350180503
SOCIAL : 2.6624548736462095
TRAVEL_AND_LOCAL : 2.33528880866426
SHOPPING : 2.2450361010830324
BOOKS_AND_REFERENCE : 2.1435018050541514
DATING : 1.861462093862816
VIDEO_PLAYERS : 1.7937725631768955
MAPS_AND_NAVIGATION : 1.3989169675090252
FOOD_AND_DRINK : 1.2409747292418771
EDUCATION : 1.1620036101083033
ENTERTAINMENT : 0.9589350180505415
LIBRARIES_AND_DEMO : 0.9363718411552346
AUTO_AND_VEHICLES : 0.9250902527075812
HOUSE_AND_HOME : 0.8235559566787004
WEATHER : 0.8009927797833934
EVENTS : 0.7107400722021661
PARENTING : 0.6543321299638989
ART_AND_DESIGN : 

The landscape seems significantly different on Google Play: there are not that many apps designed for fun, and it seems that a good number of apps are designed for practical purposes (family, tools, business, lifestyle, productivity, etc.). However, if we investigate this further, we can see that the family category (which accounts for almost 19% of the apps) means mostly games for kids.

Even so, practical apps seem to have a better representation on Google Play compared to App Store. This picture is also confirmed by the frequency table we see for the Genres column:

In [16]:
display_table(google_data_clean, -4) 

Tools : 8.449909747292418
Entertainment : 6.069494584837545
Education : 5.347472924187725
Business : 4.591606498194946
Productivity : 3.892148014440433
Lifestyle : 3.892148014440433
Finance : 3.7003610108303246
Medical : 3.531137184115524
Sports : 3.463447653429603
Personalization : 3.3167870036101084
Communication : 3.2378158844765346
Action : 3.1024368231046933
Health & Fitness : 3.0798736462093865
Photography : 2.944494584837545
News & Magazines : 2.7978339350180503
Social : 2.6624548736462095
Travel & Local : 2.3240072202166067
Shopping : 2.2450361010830324
Books & Reference : 2.1435018050541514
Simulation : 2.0419675090252705
Dating : 1.861462093862816
Arcade : 1.8501805054151623
Video Players & Editors : 1.7712093862815883
Casual : 1.7599277978339352
Maps & Navigation : 1.3989169675090252
Food & Drink : 1.2409747292418771
Puzzle : 1.128158844765343
Racing : 0.9927797833935018
Role Playing : 0.9363718411552346
Libraries & Demo : 0.9363718411552346
Auto & Vehicles : 0.9250902527075

The difference between the Genres and the Category columns is not crystal clear, but one thing we can notice is that the Genres column is much more granular (it has more categories). We're only looking for the bigger picture at the moment, so we'll only work with the Category column moving forward.

Up to this point, we found that the App Store is dominated by apps designed for fun, while Google Play shows a more balanced landscape of both practical and for-fun apps. Now we'd like to get an idea about the kind of apps that have most users.

## Most popular apps by genre

In [64]:
def get_avg_rating(dataset, installs_index, genre_index):
    genre_freq_table = freq_table(dataset, genre_index)
    
    for genre in genre_freq_table:
        total = 0
        len_genre = 0

        for row in dataset:
            genre_app = row[genre_index]
            if genre == genre_app:
                installs = row[installs_index]
                # Remove + and ,  in android dataset (installs column)
                installs = installs.replace("+", "")
                installs = installs.replace(",", "")
                installs = float(installs)
                total += installs
                len_genre += 1
        avg_installs = total/len_genre
        print(genre, ":", avg_installs)

In [66]:
print("*****APPLE*****")
get_avg_rating(apple_data_clean, 6, -5)
print("*****ANDROID*****")
get_avg_rating(google_data_clean, 5, 1)

*****APPLE*****
Productivity : 21028.410714285714
Weather : 52279.892857142855
Shopping : 26919.690476190477
Reference : 74942.11111111111
Finance : 31467.944444444445
Music : 57326.530303030304
Utilities : 18684.456790123455
Travel : 28243.8
Social Networking : 71548.34905660378
Sports : 23008.898550724636
Health & Fitness : 23298.015384615384
Games : 22812.92467948718
Food & Drink : 33333.92307692308
News : 21248.023255813954
Book : 39758.5
Photo & Video : 28441.54375
Entertainment : 14029.830708661417
Business : 7491.117647058823
Lifestyle : 16485.764705882353
Education : 7003.983050847458
Navigation : 86090.33333333333
Medical : 612.0
Catalogs : 4004.0
*****ANDROID*****
ART_AND_DESIGN : 1986335.0877192982
AUTO_AND_VEHICLES : 647317.8170731707
BEAUTY : 513151.88679245283
BOOKS_AND_REFERENCE : 8767811.894736841
BUSINESS : 1712290.1474201474
COMICS : 817657.2727272727
COMMUNICATION : 38456119.167247385
DATING : 854028.8303030303
EDUCATION : 1833495.145631068
ENTERTAINMENT : 11640705.8

# Conclusion

In this project, we analyzed data about the App Store and Google Play mobile apps with the goal of recommending an app profile that can be profitable for both markets.

The conclusion is that taking a popular book (perhaps a more recent book) and turning it into an app could be profitable for both the Google Play and the App Store markets. The markets are already full of libraries, so we need to add some special features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes on the book, a forum where people can discuss the book, etc.